In [1]:
import numpy as np
import pandas as pd
import os
%matplotlib inline

#makes the file too heavy - comment out first
#from plotly import __version__
#import cufflinks as cf
#from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
#init_notebook_mode(connected=True)
#cf.go_offline()


#user specify instrument and contract month

product=''
month=''

while True:
    
    inst=int(raw_input('Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: '))
    
    if inst==1:
        product='B'
        break
    elif inst==2:
        product='G'
        break   
    elif inst==3:
        product='HO'
        break      
    elif inst==4:
        product='RB'
        break       
    elif inst==5:
        product='CL'
        break       
    else:
        print 'You did not input a correct number. Please try again.'

while True:
    
    Mth=int(raw_input('Please input the month of the contract which you wish to evaluate in numbers from 1-12: '))
    
    if Mth==1:
        month='Jan'
        break
    elif Mth==2:
        month='Feb'
        break   
    elif Mth==3:
        month='Mar'
        break   
    elif Mth==4:
        month='Apr'
        break   
    elif Mth==5:
        month='May'
        break          
    elif Mth==6:
        month='Jun'
        break   
    elif Mth==7:
        month='Jul'
        break   
    elif Mth==8:
        month='Aug'
        break   
    elif Mth==9:
        month='Sep'
        break   
    elif Mth==10:
        month='Oct'
        break   
    elif Mth==11:
        month='Nov'
        break   
    elif Mth==12:
        month='Dec'
        break 
    else:
        print 'You did not input a correct number. Please try again.'

            



Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: 1
Please input the month of the contract which you wish to evaluate in numbers from 1-12: 3


In [2]:
cd Quandl

C:\Users\boonsin.teo\Documents\Quandl


In [3]:
cd $product

C:\Users\boonsin.teo\Documents\Quandl\B


In [4]:
cd $month

C:\Users\boonsin.teo\Documents\Quandl\B\Mar


In [5]:
#create two lists to store the start dates and end dates - later to rename the multi-index table
sd_list=[]
ed_list=[]

#indicate the days forward/backward to loop
days_to_loop_start=int(raw_input('Please indicate the number of days to loop forward/backward on start date: '))
days_to_loop_end=int(raw_input('Please indicate the number of days to loop forward/backward on end date: '))

#used to loop - starting value
loop_start=-days_to_loop_start
loop_end=-days_to_loop_end

#indicate evaluation dates

start_mth=raw_input('Please input start mth of seasonal period in MM format: ')
start_day=raw_input('Please input start day of seasonal period in DD format: ')

i=int(raw_input('Please input 1 if seasonal period start on same year as contract year and 0 if not: '))
            
end_mth=raw_input('Please input end mth of seasonal period in MM format: ')
end_day=raw_input('Please input end day of seasonal period in DD format: ')

j=int(raw_input('Please input 1 if seasonal period ends on same year as contract year and 0 if not: '))

pos=1

while True:

    a=raw_input("Please input 'y' if position is long and 'n' if position is short: ").lower()
    
    if a=='y':
        break
    if a=='n':
        pos=pos*(-1)
        break
    else:
        print "Please input either 'y' or 'n' for position"

print 'PnL will be calculated basis 1kb lot size for standardization purposes.'


outside=[a for a in range(loop_start,-loop_start+1)]
len_outside=len(outside)
outside=outside*len_outside
outside.sort()
inside=[a for a in range(loop_end,-loop_end+1)]
inside=inside*len_outside
hier_index=list(zip(outside, inside))
hier_index=pd.MultiIndex.from_tuples(hier_index)

years_list=[]

path = os.getcwd()
listing = os.listdir(path)

for infile in listing: # get list of all years to build dataframe
    
    rd_file=pd.read_excel(infile) #read each file into panda dataframe
    yr_str=infile[-8:-4] # get the contract year
    years_list.append(yr_str)

Seasonal_Combi=pd.DataFrame(np.random.randn(len(inside),len(years_list)),hier_index,years_list) 
#Multi index dataframe created. now to replace the values

first_index=0
second_index=0

while first_index<=days_to_loop_start*2: # loop for start dates
    second_index=0
    loop_end=-days_to_loop_end
    
    while second_index<=days_to_loop_end*2: #loop for end dates
        
 
        for infile in listing:

            temp=pd.read_excel(infile) #read each file into panda dataframe

            contract_yr_str=infile[-8:-4] # get the contract year

            seasonal_year=int(contract_yr_str)-1 # get the seasonal year to create the index - prev year
            seasonal_year=str(seasonal_year) # convert to string

            start_date=str(int(seasonal_year)+i)+'-'+start_mth+'-'+start_day # create start date
            end_date=str(int(seasonal_year)+j)+'-'+end_mth+'-'+end_day # create end date

            dates=pd.date_range(start_date, end_date) # range of dates for date axis

            start_date_new=dates[0]+loop_start # loop first date
            end_date_new=dates[-1]+loop_end # loop end date


            #start date cannot be greater than end date
            if start_date_new<dates[-1]:

                #create a new_dates dataframe - or else dates will be changed to an array at second line
                new_dates=pd.date_range(start_date_new,end_date_new)

                new_dates=new_dates.strftime('%Y-%m-%d') #make sure no left out dates

                sample_data=np.arange(0,len(new_dates)) #just sample numbers to create dataframe

                dates_dataframe=pd.DataFrame({'Date':new_dates,'data':sample_data}) # create dates dataframe
                dates_dataframe=dates_dataframe.set_index('Date') # set index as Date to facilitate joining

                temp=temp.set_index('Date') # set index as Date to facilitate joining

                temp=dates_dataframe.join(temp) # join dataframes together on Date


                temp.reset_index(inplace=True)

                temp=temp[temp['Date']>=start_date_new] # filter for start date

                temp=temp[temp['Date']<=end_date_new] # filter for end date

                temp=temp.set_index('Date') # creating new index as Date

                temp=pd.DataFrame(temp['Settle'],columns=['Settle']) 


                #fillna values with values of forward value
                temp.fillna(method='bfill',inplace=True)
                #fillna values with values of back value
                temp.fillna(method='ffill',inplace=True)

                #PnL - use last value vs first value

                z=1
                factor=1000
                if product =='RB':
                    z=42
                elif product=='HO':
                    z=42
                elif product=='G':
                    factor=100
                else:
                    z=1
                    factor=1000

                pnl=(temp.iloc[-1]-temp.iloc[0])*z*pos*factor 

                #put inside created table
                Seasonal_Combi.loc[loop_start].loc[loop_end][str(int(seasonal_year)+i)]=pnl


            else:
                print 'Start date cannot be greater than end date. Loop dates end one day before end date.'
                break
            
        loop_end+=1
        second_index+=1
        ed_list.append(end_date_new.strftime('%m-%d'))
            
    loop_start+=1
    first_index+=1 #looping the first index
    sd_list.append(start_date_new.strftime('%m-%d'))

Seasonal_Combi.index.names=['Start', 'End']
Seasonal_Combi.index.set_levels([sd_list,ed_list],inplace=True)
    

Please indicate the number of days to loop forward/backward on start date: 2
Please indicate the number of days to loop forward/backward on end date: 2
Please input start mth of seasonal period in MM format: 01
Please input start day of seasonal period in DD format: 02
Please input 1 if seasonal period start on same year as contract year and 0 if not: 1
Please input end mth of seasonal period in MM format: 01
Please input end day of seasonal period in DD format: 08
Please input 1 if seasonal period ends on same year as contract year and 0 if not: 1
Please input 'y' if position is long and 'n' if position is short: y
PnL will be calculated basis 1kb lot size for standardization purposes.


In [6]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl\B


In [7]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl


In [8]:
cd ..

C:\Users\boonsin.teo\Documents


In [9]:
Seasonal_Combi.head()

1996   1997    1998    1999    2000   2001    2002    2003  \
Start End                                                                  
12-31 01-06  400.0  850.0  -880.0   940.0  -620.0  730.0  1860.0  1130.0   
      01-07  400.0  720.0 -1030.0   940.0 -1040.0  730.0  1630.0   580.0   
      01-08  500.0  950.0  -910.0  1010.0 -1040.0 -130.0  1420.0   160.0   
      01-09  140.0  760.0 -1140.0  1010.0 -1040.0  240.0   600.0  1150.0   
      01-10  -30.0  600.0 -1140.0  1010.0  -520.0  860.0   770.0   940.0   

               2004    2005   ...      2009    2010    2011    2012    2013  \
Start End                     ...                                             
12-31 01-06   730.0  1980.0   ...    4450.0  3840.0  -300.0  1280.0   210.0   
      01-07   620.0  2120.0   ...    -150.0  3320.0 -1490.0  1280.0   530.0   
      01-08   840.0  2120.0   ...   -1270.0  3290.0 -1490.0  1280.0  1130.0   
      01-09  1060.0  2120.0   ...   -1580.0  3290.0 -1490.0   790.0   980.0   
      01-10  1060.0  1790.0   ...   -1580.0  3290.0   820.0  1430.0  1230.0   

               2014    2015    2016    2017    2018  
Start End                                            
12-31 01-06 -4110.0 -6080.0 -3040.0   280.0  1050.0  
      01-07 -3540.0 -6040.0 -3580.0   280.0  1050.0  
      01-08 -3820.0 -6180.0 -3740.0   280.0  1210.0  
      01-09 -4570.0 -6910.0 -3740.0 -1880.0  2250.0  
      01-10 -3920.0 -6910.0 -3740.0 -3180.0  2630.0  

[5 rows x 23 columns]

In [71]:
Win_Percent=(Seasonal_Combi[Seasonal_Combi>0].count(axis=1))/(Seasonal_Combi.count(axis=1))

In [78]:
Win_Percent.sort_values(ascending=False)

Start  End  
12-31  01-06    0.695652
       01-07    0.652174
01-02  01-06    0.608696
12-31  01-08    0.608696
       01-09    0.608696
01-01  01-06    0.608696
01-03  01-10    0.565217
12-31  01-10    0.565217
01-03  01-08    0.521739
01-01  01-07    0.521739
01-02  01-07    0.521739
01-01  01-08    0.478261
       01-09    0.478261
       01-10    0.478261
01-02  01-08    0.478261
       01-09    0.478261
       01-10    0.478261
01-04  01-08    0.478261
       01-09    0.434783
01-03  01-06    0.434783
       01-07    0.434783
       01-09    0.391304
01-04  01-06    0.391304
       01-07    0.391304
       01-10    0.391304
dtype: float64

In [79]:
pwd

u'C:\\Users\\boonsin.teo\\Documents'